PAPER: Cloud Database Encryption Technology Based on Combinatorial Encryption
            

Encrypting the data while inserting it into the mysql database and also
performing some operationson that data with the help of Combinatorial Encryption

Name: Vedant Sarnobat

Roll No.: 1911117

Batch: B4

ALL IMPORTS

In [131]:
# connecting to mysql database
import mysql.connector

from random import randint, choice
import libnum
import sys

import time

# DBMS LAYER

ciphertext database

In [132]:
# ciphertext database
config1 = {
        'user': 'root', 
        'password': 'H5@!JV@QSGerX&ZT',
        'host': 'localhost',
        'database': 'homomor_encyptionDB',
        'port': 3306,
        'raise_on_warnings': True,
    }

# Connect to DB
cipherDb = mysql.connector.connect(**config1)

plaintext database

In [133]:
# plaintext database
config2 = {
        'user': 'root', 
        'password': 'H5@!JV@QSGerX&ZT',
        'host': 'localhost',
        'database': 'plain_text',
        'port': 3306,
        'raise_on_warnings': True,
    }

# Connect to DB
plainDb = mysql.connector.connect(**config2)

# PROXY LAYER

In [134]:
# retrieve from the database

def getData(db):
    # Create Cursor
    cur = db.cursor()
    
    # select the table
    cur.execute("SELECT * FROM `table1`")
    
    # fetch all data from the table
    data = cur.fetchall()
    
    # End Session
    cur.close()
    
    # return the data 
    return data


# transfer data to the server (database)
def addData(val1, val2, db):
    # Create Cursor
    c = db.cursor()
    
    try:
        # Query
        query = ("INSERT INTO table1 VALUES (%s,%s);")
        send = (val1, val2)
        
        # Execute the query
        c.execute(query,send)
        
        # Commit the changes
        db.commit()
    except Exception as e:
        print(e)

    # End Session
    c.close()
    
    print('Sent To DB', db,'!\n')

QUERIES

In [135]:
# Query on cipher database

# Print sum of row items of all columns (eg. select sum(row1) from table1;)
def ciphQuery1():
    print("\n********************************************\
    \nExecuting Query1 in Cipher Text DB")
    
    # generating the keys
    public, private = keyGenPal()
    
    # Create Cursor
    q1 = cipherDb.cursor()
    
    # query
    q1.execute("select (col1 * col2) as Row_Sum from `homomor_encyptiondb`.`table1`;")
    
    # fetch all data from the selection
    data = q1.fetchall()
    
    # End Session
    q1.close()
    
    text = []
    dec = []
    t = []

    for col in data:
        for row in col:
#             print(y)
            # storing encrypted data
            text.append(row)
            # storing the decrypted result
            dec.append(decryptionPal(row, private))
    
    # getting the original data stored in the db
    temp = getData(cipherDb)
    print('Cipher stored in the DB: ', temp)
    
    for c in temp:
        row = []
        for r in c:
            row.append(decryptionPal(r, private))
        t.append(row)
    
    
    print('\nDiciphered Data:\t', t)
    print('\nResult (encrypted):\n', text)
    print('\nResult (decrypted):\n', dec)
    print("\n********************************************\n")

    
# Print sum of col items (after multiplying the row items)
def ciphQuery2():
    print("\n********************************************\
    \nExecuting Query2 in Cipher Text DB")

    # updating the cipher by encrypting using RSA encryption
    # Create Cursor
    q1 = cipherDb.cursor()
    
    # get data from db
    data = getData(cipherDb)
    print('data before: ', data)
    # generating keys for both encryption techniques
    palPub, palPri = keyGenPal()
    RSApub, RSApri = keyGenRSA()
    
    query = ''
    
    for c in data:
        # deciphering
        r1 = decryptionPal(c[0], palPri)
        r2 = decryptionPal(c[1], palPri)
        
        # encrypting data using RSA technique
        c1 = encryptionRSA(r1, RSApub)
        c2 = encryptionRSA(r2, RSApub)
        
        # writing the query
#         query = query + "\
#             UPDATE `homomor_encyptiondb`.`table1`\
#              SET `col1` = "+ str(c1) +",\
#               `col2` = "+ str(c2) +"\
#              WHERE `col1`="+ str(c[0]) +"\
#                  AND `col2` = "+ str(c[1]) +";"
        q1.callproc('updateTable', [c1, c2, c[0], c[1]])
        
    # query execution
    q1.execute(query)
    print('Query executed for updating the cipher(to RSA)\n')
    
    # End Session
#     q1.close
    
    # start new cursor
#     q2 = cipherDb.cursor()
    
    # query
    q1.execute("select (col1 * col2) as col3 from `homomor_encyptiondb`.`table1`")
        
    # fetch all data from the selection
    data = q1.fetchall()
    print('data after: ', data)

    # End Session
#     q2.close
    
    total = 0
    result = []
    for col in data:
        for row in col:
            product = decryptionRSA(row, RSApub)
            print('sum:', total, 'product:', product)
            total = total + product
    
    print('\nResult: ', total)
    
    
    query = ''
    data = getData(cipherDb)
    # goin back to the previous encryption
    for c in data:
        # deciphering RSA
        r1 = decryptionRSA(c[0], RSApri)
        r2 = decryptionRSA(c[1], RSApri)
        
        # encrypting data using Paillier technique
        c1 = encryptionPal(r1, palPub)
        c2 = encryptionPal(r2, palPub)
        
        # writing the query
#         query = query + "\
#             UPDATE `homomor_encyptiondb`.`table1`\
#              SET `col1` = "+ str(c1) +",\
#               `col2` = "+ str(c2) +"\
#              WHERE `col1`="+ str(c[0]) +"\
#                  AND `col2` = "+ str(c[1]) +";"
        q1.callproc('updateTable', [c1, c2, c[0], c[1]])

        
    # new cursor
#     q3 = cipherDB.cursor()
    # query execution
    q1.execute(query)
    print('Query executed for updating the cipher (back to Paillier)\n')
    # End Session
    q1.close
    
    data = getData(cipherDb)
    print('data at last: ', data)
    
    print("********************************************\n")

In [136]:
# query1()
# cipherDb.close()
# ciphQuery2()

In [137]:
# Query on plain text database

# Print sum of row items of all columns (eg. select sum(row1) from table1;)
def plainQuery1():
    print("\n********************************************\
    \nExecuting Query1 in Plain Text DB")

    # Create Cursor
    q1 = plainDb.cursor()
    
    # query
    q1.execute("select (col1 + col2) as Row_Sum from `plain_text`.`table1`;")
    
    # fetch all data from the selection
    data = q1.fetchall()

    # End Session
    q1.close()
        
    temp = getData(plainDb)
    t=[]
    for x in temp:
        p=[]
        for y in x:
            p.append(y)
        t.append(p)
        
    print('Original Data:', t,'\nResult:', data)
    print("\n********************************************\n")
    
# Print sum of col items (after multiplying the row items)
def plainQuery2():
    print("\n********************************************\
    \nExecuting Query2 in Plain Text DB")

    # Create Cursor
    q2 = plainDb.cursor()
    
    # query
    q2.execute("select sum(Product) from (select col1 * col2 as Product from `plain_text`.`table1`) as Result;")
    
    # fetch all data from the selection
    data = q2.fetchall()

    print("The query selected is:\nselect sum(Product) from (select col1 * col2 as Product from `plain_text`.`table1`) as Result;\n")
    print('Result is:', data)
    # End Session
    q2.close()
    
    print("\n********************************************\n")

# APPLICATION LAYER

ALL NECESSARY FUNCTIONS

In [138]:
# check if number is prime or not
def isPrime(x):
    count = 0
    for i in range(int(x/2)):
        if x % (i+1) == 0:
            count = count+1
    return count == 1

# Compute the greatest common divisor of a and b
def gcd(a,b):    
    while b > 0:
        a, b = b, a % b
    return a
    
# Compute the lowest common multiple of a and b
def lcm(a, b):
    return a * b // gcd(a, b)

# Find the common prime divisors
def noCommon_prime(a, b) :
    
    # Get the GCD of the given numbers
    __gcd = gcd(a, b)
      
    # Find the prime divisors of the gcd
    for i in range(2, __gcd + 1) :
    
    # If i is prime and a divisor of gcd
        if prime[i] and __gcd % i == 0 :
            return False
    return True

prime = [i for i in range(1,1000) if isPrime(i)]

RSA ENCRYPTION

In [139]:
# rsa encryption

#     data = dict.fromkeys(['text', 'public', 'private'])

# function for key generation
def keyGenRSA():
    
    # Select two different large prime numbers p,q. 
#     p = random.choice(primes)
#     q = random.choice(primes)
#     while (p == q):
#         q = random.choice(primes)

    p = 19
    q = 17

    # Set n = p.q
    n = p * q

    # Calculate its Euler function r = (p − 1) ∙ (q − 1).
    r = (p - 1) * (q - 1)

    # Randomly select an integer e to meet 1 < e < r, and gcd(e, r) = 1.
    e = 181
#     e = randint(1, r)
#     while (gcd(e, r) != 1) or noCommon_prime(e, r)==False:
#         e = randint(1, r)

    # Calculate the modular inverse:  d = e^-1 mod r.
    d = libnum.invmod(e, r)

    # display the details
    print("\nRSA KEY GENERATION:\n================\np =",p,"\tq =",q)
    print("n =", n, "\nr =", r)
    print("\n================\n")
    print("e:", e,"\td:", d)
    print("\n================\n")
    print("Public key (n,e):\t", n, ",", e)
    print("Private key (n,d):\t", n, ",", d)
    print("\n================\n")

    # The public key is (n, e)
    # The private key is (n, d)
#     print("public key: ",n, ", ", e)
#     print("private key: ",n, ", ", d)
    return [n, e], [n, d]


# function for encryption
def encryptionRSA(mesg, public):
    # get the public key
    n, e = public
    
#     mesg = int(input("Enter m: "))
#     mesg = 44
    
    # encryption: c = m^e mod n
    ciph = pow(mesg, e, n)
#     print("cipher text: ", ciph)
    
    # return the encrypted message
    return ciph


# function for decryption
def decryptionRSA(ciph, private):
    # get the private key
    n, d = private
    
    # decryption m = c^d mod n
    dec = pow(ciph, d, n)
#     print("decrypted text: ", dec)
    
    # return the decrypted message
    return dec

In [140]:
# # DEBUGGING
# pub, pri = keyGenRSA()
# m1 = 15#randint(1, 50)
# m2 = 10#randint(1, 50)
# print(m1, m2)
# c1 = encryptionRSA(m1, pub)#[323 , 9])
# c2 = encryptionRSA(m2, pub)#[323 , 9])
# c = c1 * c2
# print('c:', c)
# dec = decryptionRSA(c, pri)#[144 , 126, 323])
# print(m1*m2, 'ans:', dec)

PAILLIER ENCRYPTION

In [141]:
# paillier encryption

# division on the rational domain
def L(x,n):
    return ((x-1) // n)


# function for key generation
def keyGenPal():

    # Selecting two distinct large prime numbers p,q. 
    p = 17
    q = 19

#     p = choice(primes)
#     q = choice(primes)
#     while (p == q):
#         q = choice(primes)

    # Set n = p.q
    n = p * q

    # Calculate λ = lcm of p-1 and q-1
    gLambda = lcm(p - 1, q - 1)

    # Randomly select an integer g which is a perfect square between 1 & n^2
#     g = randint(20, 150)#(1, n*n)
    g = 150
#     g = g * g
    
    # calculate mu
    l = (pow(g, gLambda, n * n) - 1) // n
    
    # check if no prime factor exsists
    while noCommon_prime(l,n) == False:
        print('*\nproblem found\n*\n')
        g = randint(20, 150)#(1, n*n)
        l = (pow(g, gLambda, n * n) - 1) // n
        
    gMu = libnum.invmod(l, n)
    
    # display the details
    print("\nPaillier KEY GENERATION:\n================\np=",p,"\tq=",q)
    print("g=",g)
    print("\n================\n")
    print("Mu:",gMu,"\tgLambda:",gLambda)
    print("\n================\n")
    print("Public key (n,g):\t\t", n, ",", g)
    print("Private key (lambda,mu):\t", gLambda, ",", gMu)
    print("\n================\n")
    
    # returning the public and the private keys
    # public key: n, g
    # private key: lambda, mu
    return [n, g], [gLambda, gMu, n]


# function for encryption
def encryptionPal(msg, public):

    # get the public key
    n, g = public
    
    print("Message:\t", msg)

    # randomly select an integer r
#     r = randint(20, 150)
    r = 48

    # g ^ msg
    k1 = pow(g, msg, n * n)
    # r ^ n
    k2 = pow(r, n, n * n)

    #cipher text
    cipher = (k1 * k2) % (n * n)
    
    # return the cipher text
    return cipher


# function for decryption
def decryptionPal(cipher, private):

    # get the private key
    gLambda, gMu, n = private
    
    l = (pow(cipher, gLambda, n * n) - 1) // n

    mess= (l * gMu) % n
    
    # return the decrypted message
    return mess

In [142]:
# # debugging
# pub, pri = keyGenPal()
# m1 = randint(1, 50)
# m2 = randint(1, 50)
# print(m1, m2)
# c1 = encryptionPal(m1, pub)#[323 , 9])
# c2 = encryptionPal(m2, pub)#[323 , 9])
# c = c1 * c2
# print('c:', c)
# dec = decryptionPal(c, pri)#[144 , 126, 323])
# print(m1+m2, 'ans:', dec)
# # dec2 = decryptionPal(c2, pri)

In [143]:
def inputData():
    # taking the input
    e1 = int(input("1st number: "))
    e2 = int(input("2nd number: "))
    
    # generating the keys
    public , private = keyGenPal()
    print ("public: ", public, "private: ", private)
    
    # generating the cipher text  
    c1 = encryptionPal(e1, public)
    c2 = encryptionPal(e2, public)
    print("c1: ", c1, "\tc2: ", c2)
    
    # sending data to the both the databases
    addData(c1, c2, cipherDb)
    addData(e1, e2, plainDb)
    
    mesg1 = decryptionPal(c1, private)
    mesg2 = decryptionPal(c2, private)
    print("deciph: ", mesg1, mesg2)
    
    
# calculating the time taken for execution of the queries
def query1():
    # plain text db
    start = time.time()
    plainQuery1()
    end = time.time()
    print('Time taken for 1st query in Plain Text database::', end - start)
    
    # cipher text db
    start = time.time()
    ciphQuery1()
    end = time.time()
    print('Time taken for 1st query in Cipher Text database:', end - start)
    
def query2():
    # plain text db
    start = time.time()
    plainQuery2()
    end = time.time()
    print('Time taken for 2nd query in Plain Text database::', end - start)
    
    # cipher text db
    start = time.time()
    ciphQuery2()
    end = time.time()
    print('Time taken for 2nd query in Cipher Text database:', end - start)
    

def exiting():
    # closing all the database connections
    cipherDb.close()
    plainDb.close()
    

    
# main 
if __name__ == '__main__':
    
    print("PAPER: Cloud Database Encryption Technology Based on Combinatorial Encryption\
    \n\nEncrypting the data while inserting it into the mysql database and also\
    \nperforming some operationson that data with the help of Combinatorial Encryption")
    
    switcher = {
        1: inputData,
        2: query1,
        3: query2,
        0: exiting
    }
    
    opt = -1
    
    while opt!= 0:
        print("\nOptions:\
            \n1. Insert new row\
            \n2. Print sum of row items of all columns (select sum(row1) from table1;)\
            \n3. Print sum of col items (after multiplying the row items)\
            \n   (select sum(row3) from (select row1*row2 as row3 from table1);)\
            \n0. Exit")
        
        opt = int(input("Enter your option: "))

        # switch case
        try:
            switcher[opt]()
        except KeyError:
            print("Enter a valid option!")

    print('Thank You!')

PAPER: Cloud Database Encryption Technology Based on Combinatorial Encryption    

Encrypting the data while inserting it into the mysql database and also    
performing some operationson that data with the help of Combinatorial Encryption

Options:            
1. Insert new row            
2. Print sum of row items of all columns (select sum(row1) from table1;)            
3. Print sum of col items (after multiplying the row items)            
   (select sum(row3) from (select row1*row2 as row3 from table1);)            
0. Exit
Enter your option: 1
1st number: 2
2nd number: 4

Paillier KEY GENERATION:
p= 17 	q= 19
g= 150


Mu: 135 	gLambda: 144


Public key (n,g):		 323 , 150
Private key (lambda,mu):	 144 , 135


public:  [323, 150] private:  [144, 135, 323]
Message:	 2
Message:	 4
c1:  19103 	c2:  86349
Sent To DB <mysql.connector.connection_cext.CMySQLConnection object at 0x00000281B421A9D0> !

Sent To DB <mysql.connector.connection_cext.CMySQLConnection object at 0x00000281B421A52

Enter your option: 0
Thank You!


In [144]:
# cipherDB.close()
# plainDB.close()